# Ch13: 作用域與生命週期 - 詳解範例

本 Notebook 包含 **4 個完整的實務案例**，展示作用域與閉包的實際應用。

## 📋 案例清單
1. 閉包實作：銀行帳戶系統（含交易記錄進階版）
2. 裝飾器計時器：使用閉包統計呼叫次數
3. LEGB 規則除錯：診斷並修正 UnboundLocalError
4. 迴圈中的閉包陷阱：延遲綁定問題 + 3 種解法

---

## 案例 1: 閉包實作：銀行帳戶系統（進階版）

### 情境
實作一個銀行帳戶系統，使用閉包封裝帳戶餘額和交易記錄。需要提供存款、提款、查詢餘額、查詢交易記錄等功能。

### 學習重點
- 使用閉包封裝私有狀態（餘額、交易記錄）
- nonlocal 關鍵字修改外層變數
- 回傳多個內層函式組成的介面
- 資料安全性：外部無法直接修改餘額

### 分析思路
1. 外層函式 `create_account()` 定義私有變數（balance, transactions）
2. 內層函式使用 `nonlocal` 修改這些變數
3. 回傳 dict 包含所有操作函式（類似物件的方法）
4. 每次呼叫 `create_account()` 創建獨立的帳戶

### 完整程式碼

In [ ]:
def create_account(initial_balance=0, account_name="Unknown"):
    """
    建立銀行帳戶（使用閉包封裝）
    
    參數:
        initial_balance (float): 初始餘額
        account_name (str): 帳戶名稱
    
    回傳:
        dict: 包含各操作函式的介面
    """
    # 私有變數（外部無法直接訪問）
    balance = initial_balance
    transactions = []
    
    # 記錄初始餘額
    if initial_balance > 0:
        transactions.append({
            'type': '開戶',
            'amount': initial_balance,
            'balance': balance
        })
    
    def deposit(amount):
        """存款"""
        nonlocal balance  # 宣告要修改外層變數
        
        if amount <= 0:
            return f"❌ 存款金額必須大於 0"
        
        balance += amount
        transactions.append({
            'type': '存款',
            'amount': amount,
            'balance': balance
        })
        
        return f"✅ 存款 ${amount:.2f} 成功，餘額：${balance:.2f}"
    
    def withdraw(amount):
        """提款"""
        nonlocal balance
        
        if amount <= 0:
            return f"❌ 提款金額必須大於 0"
        
        if amount > balance:
            return f"❌ 餘額不足！當前餘額：${balance:.2f}"
        
        balance -= amount
        transactions.append({
            'type': '提款',
            'amount': amount,
            'balance': balance
        })
        
        return f"✅ 提款 ${amount:.2f} 成功，餘額：${balance:.2f}"
    
    def get_balance():
        """查詢餘額"""
        return f"帳戶 [{account_name}] 餘額：${balance:.2f}"
    
    def get_transactions():
        """查詢交易記錄"""
        if not transactions:
            return "尚無交易記錄"
        
        result = f"帳戶 [{account_name}] 交易記錄：\n"
        for i, trans in enumerate(transactions, 1):
            result += f"  {i}. {trans['type']}: ${trans['amount']:.2f} (餘額: ${trans['balance']:.2f})\n"
        return result
    
    # 回傳介面（類似物件的公開方法）
    return {
        'deposit': deposit,
        'withdraw': withdraw,
        'get_balance': get_balance,
        'get_transactions': get_transactions
    }

# 測試 1：建立帳戶並進行基本操作
print("=== 測試 1: 基本操作 ===")
alice_account = create_account(1000, "Alice")

print(alice_account['get_balance']())
print(alice_account['deposit'](500))
print(alice_account['withdraw'](200))
print(alice_account['get_balance']())

print("\n" + "="*50 + "\n")

# 測試 2：錯誤處理
print("=== 測試 2: 錯誤處理 ===")
print(alice_account['withdraw'](2000))  # 餘額不足
print(alice_account['deposit'](-100))   # 無效金額

print("\n" + "="*50 + "\n")

# 測試 3：交易記錄
print("=== 測試 3: 交易記錄 ===")
print(alice_account['get_transactions']())

print("\n" + "="*50 + "\n")

# 測試 4：獨立性測試（多個帳戶）
print("=== 測試 4: 獨立性測試 ===")
bob_account = create_account(500, "Bob")

print(alice_account['get_balance']())
print(bob_account['get_balance']())

print("\nAlice 存款 $300:")
print(alice_account['deposit'](300))

print("\nBob 提款 $100:")
print(bob_account['withdraw'](100))

print("\n檢查餘額（互不影響）：")
print(alice_account['get_balance']())
print(bob_account['get_balance']())

### 知識點總結

#### 1. 閉包封裝私有狀態
```python
# balance 和 transactions 是「私有變數」
# 外部無法直接訪問或修改
balance = initial_balance
transactions = []

# ❌ 外部無法這樣做：
# alice_account['balance'] = 10000  # 沒有這個 key
```

#### 2. nonlocal 關鍵字的必要性
```python
# ❌ 錯誤：沒有 nonlocal 會導致 UnboundLocalError
def deposit(amount):
    balance += amount  # UnboundLocalError!

# ✅ 正確：使用 nonlocal 宣告
def deposit(amount):
    nonlocal balance
    balance += amount
```

#### 3. 閉包的獨立性
- 每次呼叫 `create_account()` 都創建新的 `balance` 和 `transactions`
- 不同帳戶之間完全獨立，互不影響

#### 4. 閉包 vs 類別
| 特點 | 閉包 | 類別 |
|:-----|:-----|:-----|
| 語法複雜度 | 簡單 | 較複雜 |
| 資料封裝 | 天然私有 | 需要命名規則（_private） |
| 擴展性 | 較差 | 較好（繼承） |
| 適用場景 | 簡單狀態保持 | 複雜物件設計 |

---

## 案例 2: 裝飾器計時器（閉包應用）

### 情境
設計一個裝飾器，用於統計函式的呼叫次數和總執行時間。

### 學習重點
- 閉包在裝飾器中的應用
- 使用閉包保存狀態（呼叫次數、總時間）
- nonlocal 修改外層變數
- 函式作為回傳值

### 分析思路
1. 裝飾器本質上就是一個閉包
2. 外層函式接收被裝飾的函式
3. 內層函式（wrapper）執行實際邏輯並統計資訊
4. 使用閉包變數保存統計資料

### 完整程式碼

In [ ]:
import time

def call_counter(func):
    """
    裝飾器：統計函式呼叫次數和總執行時間
    
    參數:
        func: 被裝飾的函式
    
    回傳:
        wrapper: 包裝後的函式
    """
    # 閉包變數：保存統計資訊
    count = 0
    total_time = 0.0
    
    def wrapper(*args, **kwargs):
        """包裝函式（內層函式）"""
        nonlocal count, total_time  # 宣告要修改外層變數
        
        # 呼叫次數 +1
        count += 1
        
        # 記錄開始時間
        start_time = time.time()
        
        # 執行原始函式
        result = func(*args, **kwargs)
        
        # 計算執行時間
        elapsed_time = time.time() - start_time
        total_time += elapsed_time
        
        # 顯示統計資訊
        print(f"[統計] {func.__name__}() 第 {count} 次呼叫")
        print(f"       本次執行時間: {elapsed_time:.4f} 秒")
        print(f"       累計執行時間: {total_time:.4f} 秒")
        print(f"       平均執行時間: {total_time/count:.4f} 秒\n")
        
        return result
    
    # 回傳包裝後的函式
    return wrapper

# 測試 1：裝飾簡單函式
@call_counter
def calculate_sum(n):
    """計算 1 到 n 的總和"""
    time.sleep(0.1)  # 模擬耗時操作
    return sum(range(1, n + 1))

print("=== 測試 1: 多次呼叫 ===")
print(f"結果: {calculate_sum(100)}\n")
print(f"結果: {calculate_sum(200)}\n")
print(f"結果: {calculate_sum(300)}\n")

print("="*50 + "\n")

# 測試 2：裝飾複雜函式
@call_counter
def fibonacci(n):
    """計算第 n 個費波那契數（遞迴版本）"""
    if n <= 1:
        return n
    return fibonacci.__wrapped__(n-1) + fibonacci.__wrapped__(n-2)

# 為了避免遞迴時重複計數，暫時儲存原始函式
fibonacci.__wrapped__ = fibonacci.__wrapped__ if hasattr(fibonacci, '__wrapped__') else lambda n: n if n <= 1 else fibonacci.__wrapped__(n-1) + fibonacci.__wrapped__(n-2)

# 重新定義不使用裝飾器的版本進行測試
@call_counter
def slow_function():
    """模擬慢速函式"""
    time.sleep(0.2)
    return "完成"

print("=== 測試 2: 慢速函式 ===")
slow_function()
slow_function()
slow_function()

### 更進階：可配置的計數器

讓裝飾器支援自訂配置（是否顯示詳細資訊）

In [ ]:
def call_counter_v2(verbose=True):
    """
    可配置的計數器裝飾器
    
    參數:
        verbose (bool): 是否顯示詳細資訊
    
    回傳:
        decorator: 裝飾器函式
    """
    def decorator(func):
        count = 0
        total_time = 0.0
        
        def wrapper(*args, **kwargs):
            nonlocal count, total_time
            
            count += 1
            start_time = time.time()
            result = func(*args, **kwargs)
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            
            if verbose:
                print(f"[{func.__name__}] 呼叫 #{count}, 耗時 {elapsed_time:.4f}s")
            
            return result
        
        # 提供查詢統計資訊的方法
        def get_stats():
            return {
                'count': count,
                'total_time': total_time,
                'average_time': total_time / count if count > 0 else 0
            }
        
        wrapper.get_stats = get_stats
        return wrapper
    
    return decorator

# 測試：靜默模式
@call_counter_v2(verbose=False)
def quiet_function(x):
    time.sleep(0.05)
    return x * 2

print("=== 測試 3: 靜默模式 ===")
for i in range(5):
    quiet_function(i)

# 查詢統計資訊
stats = quiet_function.get_stats()
print(f"\n統計資訊：")
print(f"  呼叫次數: {stats['count']}")
print(f"  總時間: {stats['total_time']:.4f} 秒")
print(f"  平均時間: {stats['average_time']:.4f} 秒")

### 知識點總結

#### 1. 裝飾器就是閉包的應用
```python
# 裝飾器結構
def decorator(func):        # 外層函式：接收原函式
    # 閉包變數
    count = 0
    
    def wrapper(*args):     # 內層函式：包裝原函式
        nonlocal count      # 修改外層變數
        count += 1
        return func(*args)  # 呼叫原函式
    
    return wrapper          # 回傳內層函式
```

#### 2. 閉包保存狀態的優勢
- 不需要全域變數
- 每個被裝飾的函式有獨立的計數器
- 狀態持久化（多次呼叫累計）

#### 3. 多層閉包（裝飾器工廠）
```python
# 層次 1：配置函式
def call_counter_v2(verbose=True):
    # 層次 2：裝飾器
    def decorator(func):
        # 層次 3：包裝函式
        def wrapper(*args):
            # 可訪問所有外層變數：verbose, func
            pass
        return wrapper
    return decorator
```

---

## 案例 3: LEGB 規則除錯 - UnboundLocalError

### 情境
診斷並修正常見的作用域錯誤，理解 Python 的 LEGB 查找規則。

### 學習重點
- 理解 UnboundLocalError 的成因
- Python 的「編譯時決策」機制
- 如何正確使用 global 和 nonlocal
- 除錯技巧：使用 locals() 和 globals()

### 分析思路
1. Python 在函式編譯時決定變數的作用域
2. 看到 `x = ...` 就判定 x 是區域變數
3. 如果在賦值前嘗試讀取，就會報錯

### 完整程式碼

In [ ]:
print("=== 錯誤案例 1: UnboundLocalError ===")

counter = 0

def increment_wrong():
    """❌ 錯誤：嘗試在賦值前讀取"""
    print(counter)  # UnboundLocalError!
    counter += 1    # Python 看到這行，判定 counter 是區域變數
    return counter

try:
    increment_wrong()
except UnboundLocalError as e:
    print(f"❌ 錯誤發生：{e}")
    print("\n原因分析：")
    print("1. Python 編譯函式時，看到 'counter += 1'")
    print("2. 判定 counter 是『區域變數』")
    print("3. 在 print(counter) 時，counter 尚未賦值")
    print("4. 導致 UnboundLocalError\n")

print("="*50 + "\n")

# 解決方法 1：使用 global
print("=== 解決方法 1: 使用 global ===")

counter = 0

def increment_global():
    """✅ 正確：使用 global 宣告"""
    global counter  # 明確宣告使用全域變數
    print(f"讀取全域變數 counter: {counter}")
    counter += 1
    return counter

print(f"第 1 次呼叫: {increment_global()}")
print(f"第 2 次呼叫: {increment_global()}")
print(f"全域變數 counter: {counter}\n")

print("="*50 + "\n")

# 解決方法 2：使用不同的變數名（推薦）
print("=== 解決方法 2: 使用不同變數名（推薦）===")

counter = 0

def increment_local():
    """✅ 正確：使用區域變數"""
    local_counter = counter  # 讀取全域變數到區域變數
    local_counter += 1
    print(f"區域變數: {local_counter}")
    print(f"全域變數: {counter} (未被修改)")
    return local_counter

increment_local()
increment_local()
print(f"全域變數 counter 始終為: {counter}\n")

print("="*50 + "\n")

### 錯誤案例 2: global vs nonlocal 的混淆

In [ ]:
print("=== 錯誤案例 2: global vs nonlocal 的混淆 ===")

x = "global x"

def outer():
    x = "enclosing x"
    
    def inner_wrong():
        """❌ 錯誤：想修改 outer 的 x，卻用了 global"""
        global x  # 這會修改全域的 x，而非 outer 的 x
        x = "modified by inner"
        print(f"inner 內部: {x}")
    
    def inner_correct():
        """✅ 正確：使用 nonlocal 修改 outer 的 x"""
        nonlocal x  # 修改外層函式的 x
        x = "modified by inner"
        print(f"inner 內部: {x}")
    
    print(f"outer 開始: {x}")
    
    print("\n使用 global (錯誤):")
    inner_wrong()
    print(f"outer 的 x: {x} (未被修改)")
    
    print("\n使用 nonlocal (正確):")
    inner_correct()
    print(f"outer 的 x: {x} (已被修改)")

outer()
print(f"\n全域的 x: {x} (被 inner_wrong 修改了)")

print("\n" + "="*50 + "\n")

### 除錯技巧：使用 locals() 和 globals()

In [ ]:
print("=== 除錯技巧：檢視變數作用域 ===")

global_var = "我是全域變數"

def debug_scope():
    local_var = "我是區域變數"
    
    print("區域變數（locals()）:")
    for name, value in locals().items():
        print(f"  {name} = {value}")
    
    print("\n全域變數（globals()，只顯示自訂的）:")
    for name, value in globals().items():
        if not name.startswith('_') and name in ['global_var']:
            print(f"  {name} = {value}")

debug_scope()

print("\n" + "="*50 + "\n")

### 知識點總結

#### 1. UnboundLocalError 的成因
```python
# Python 的「編譯時決策」
x = 10
def func():
    print(x)  # ❌ UnboundLocalError
    x = 20    # Python 看到這行，整個函式內的 x 都是區域變數
```

#### 2. 何時使用 global vs nonlocal

| 關鍵字 | 作用範圍 | 使用時機 | 範例 |
|:-------|:---------|:---------|:-----|
| **global** | 模組層級 | 修改全域變數 | `global counter` |
| **nonlocal** | 外層函式 | 修改外層函式變數 | `nonlocal count` |

#### 3. 最佳實踐
1. **避免使用 global**：優先使用參數與回傳值
2. **明確命名**：區域變數與全域變數使用不同名稱
3. **除錯工具**：使用 `locals()` 和 `globals()` 檢視變數

---

## 案例 4: 迴圈中的閉包陷阱（延遲綁定問題）

### 情境
在迴圈中創建閉包時，常見的「延遲綁定」問題及其解決方案。

### 學習重點
- 理解閉包的「延遲綁定」機制
- Python 的變數查找是在「執行時」而非「定義時」
- 3 種解決方案的比較

### 分析思路
1. 閉包捕獲的是「變數本身」，而非「變數的值」
2. 迴圈結束後，所有閉包共享同一個變數
3. 執行閉包時，變數已經是最後的值

### 完整程式碼

In [ ]:
print("=== 問題演示：迴圈中的閉包陷阱 ===")

# ❌ 錯誤示範
def create_multipliers_wrong():
    """創建多個乘法函式（錯誤版本）"""
    multipliers = []
    
    for i in range(1, 4):  # i = 1, 2, 3
        def multiplier(x):
            return x * i  # 捕獲變數 i
        multipliers.append(multiplier)
    
    return multipliers

funcs = create_multipliers_wrong()

print("預期：[10, 20, 30]")
print(f"實際：{[f(10) for f in funcs]}")
print("\n為什麼都是 30？")
print("1. 迴圈結束後，i = 3")
print("2. 所有閉包共享同一個變數 i")
print("3. 執行時，i 的值已經是 3")

print("\n" + "="*50 + "\n")

### 解決方案 1：使用預設參數（最推薦）

In [ ]:
print("=== 解決方案 1: 使用預設參數（推薦）===")

def create_multipliers_solution1():
    """✅ 正確：使用預設參數捕獲值"""
    multipliers = []
    
    for i in range(1, 4):
        def multiplier(x, n=i):  # n=i 在定義時就綁定了值
            return x * n
        multipliers.append(multiplier)
    
    return multipliers

funcs = create_multipliers_solution1()
print(f"結果：{[f(10) for f in funcs]}")

print("\n原理：")
print("1. 預設參數在函式『定義時』就賦值")
print("2. n=i 會在每次迴圈時捕獲當時的 i 值")
print("3. 每個函式都有獨立的 n 值")

print("\n" + "="*50 + "\n")

### 解決方案 2：使用閉包工廠（更明確）

In [ ]:
print("=== 解決方案 2: 使用閉包工廠 ===")

def create_multipliers_solution2():
    """✅ 正確：使用工廠函式"""
    multipliers = []
    
    def make_multiplier(n):
        """工廠函式：創建乘法閉包"""
        def multiplier(x):
            return x * n  # 捕獲參數 n
        return multiplier
    
    for i in range(1, 4):
        multipliers.append(make_multiplier(i))  # 每次呼叫創建新閉包
    
    return multipliers

funcs = create_multipliers_solution2()
print(f"結果：{[f(10) for f in funcs]}")

print("\n原理：")
print("1. 每次呼叫 make_multiplier(i) 都創建新的作用域")
print("2. 每個作用域有獨立的參數 n")
print("3. 內層 multiplier 捕獲的是不同的 n")

print("\n" + "="*50 + "\n")

### 解決方案 3：使用 lambda 配合預設參數

In [ ]:
print("=== 解決方案 3: lambda 配合預設參數 ===")

def create_multipliers_solution3():
    """✅ 正確：使用 lambda 與預設參數"""
    return [lambda x, n=i: x * n for i in range(1, 4)]

funcs = create_multipliers_solution3()
print(f"結果：{[f(10) for f in funcs]}")

print("\n原理：")
print("1. lambda 也是函式，可以使用預設參數")
print("2. n=i 在每次迴圈時綁定當時的值")
print("3. 語法更簡潔（但可讀性稍差）")

print("\n" + "="*50 + "\n")

### 實際案例：GUI 事件處理

In [ ]:
print("=== 實際案例：模擬 GUI 按鈕事件 ===")

# ❌ 錯誤示範
print("錯誤版本：")
buttons_wrong = []
for i in range(1, 4):
    # 模擬按鈕點擊事件
    def on_click():
        print(f"按鈕 {i} 被點擊")  # 都會顯示「按鈕 3」
    buttons_wrong.append(on_click)

for btn in buttons_wrong:
    btn()  # 預期：按鈕 1, 2, 3；實際：全部顯示按鈕 3

print("\n" + "-"*50 + "\n")

# ✅ 正確版本
print("正確版本：")
buttons_correct = []
for i in range(1, 4):
    def on_click(button_id=i):  # 使用預設參數
        print(f"按鈕 {button_id} 被點擊")
    buttons_correct.append(on_click)

for btn in buttons_correct:
    btn()

print("\n" + "="*50 + "\n")

### 知識點總結

#### 1. 延遲綁定問題的本質
```python
# 閉包捕獲的是「變數引用」而非「變數值」
for i in range(3):
    def f():
        return i  # 捕獲的是變數 i，而非 i 的值

# 所有 f() 都共享同一個 i
# 執行時，i 已經變成 2（迴圈最後的值）
```

#### 2. 三種解決方案比較

| 解決方案 | 語法 | 優點 | 缺點 |
|:---------|:-----|:-----|:-----|
| **預設參數** | `def f(x, n=i)` | 簡潔、高效 | 參數列表變長 |
| **閉包工廠** | `make_func(i)` | 明確、易讀 | 多一層函式 |
| **lambda** | `lambda x, n=i` | 最簡潔 | 可讀性稍差 |

#### 3. 最佳實踐
1. **推薦使用預設參數**（簡潔且高效）
2. **複雜情況用閉包工廠**（可讀性更好）
3. **避免在迴圈中直接創建閉包**

#### 4. 記憶口訣
- **閉包捕獲變數，不捕獲值**
- **迴圈閉包需小心，預設參數來解圍**

---

## 總結

本 Notebook 透過 4 個實務案例，深入探討了作用域與閉包的核心概念：

### 案例回顧

1. **銀行帳戶系統**
   - 閉包封裝私有狀態
   - nonlocal 修改外層變數
   - 閉包的獨立性

2. **裝飾器計時器**
   - 裝飾器是閉包的應用
   - 使用閉包保存狀態
   - 多層閉包（裝飾器工廠）

3. **LEGB 規則除錯**
   - UnboundLocalError 的成因
   - global vs nonlocal 的區別
   - 除錯工具：locals() 和 globals()

4. **迴圈閉包陷阱**
   - 延遲綁定問題
   - 3 種解決方案
   - 實際應用場景

### 關鍵要點

- ✅ 閉包 = 函式 + 捕獲的外層變數
- ✅ nonlocal 用於修改外層函式變數
- ✅ global 用於修改模組層級變數
- ✅ 閉包捕獲的是變數引用，而非值
- ✅ 迴圈中創建閉包需使用預設參數

### 下一步

完成本詳解範例後，請繼續：
1. **課堂練習**：`03-practice.ipynb`（12 題）
2. **課後習題**：`04-exercises.ipynb`（18 題）
3. **自我測驗**：`quiz.ipynb`（20 題）